In [21]:
from birdclef.utils import get_spark

spark = get_spark()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/11 21:02:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/11 21:02:45 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [22]:
df_sample = spark.read.csv(
    "gs://dsgt-clef-birdclef-2024/data/raw/birdclef-2024/sample_submission.csv",
    header=True,
)

In [38]:
df = spark.read.csv("/mnt/data/tmp/submission.csv", header=True)

In [39]:
assert len(df_sample.columns) > 0
assert df_sample.columns == df.columns

In [40]:
df.show(n=3)

+-------------+------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+-------+-------+-------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+------+-------+-------+-------+------+-------+-------+-------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+-------+------+-------+------+------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+------+-------+-------+------+-------+------+-------+-------+-------+------+-------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+---

In [41]:
from pyspark.sql import functions as F

df.select(F.split("row_id", "_")[0].alias("name")).groupBy("name").count().groupBy(
    "count"
).count().show()
df.select(F.split("row_id", "_")[1].alias("name")).groupBy("name").count().groupBy(
    "count"
).count().show()

+-----+-----+
|count|count|
+-----+-----+
|   48|   99|
|    8|    1|
+-----+-----+

+-----+-----+
|count|count|
+-----+-----+
|  100|    8|
|   99|   40|
+-----+-----+



In [30]:
# let's check what's going on with the submission files
import librosa
from pathlib import Path
from collections import Counter

# check the sizes of the soundscapes
soundscapes = sorted(
    Path("/mnt/data/raw/birdclef-2024/unlabeled_soundscapes").glob("**/*.ogg")
)[:100]

sizes_sec = []
for s in soundscapes:
    sizes_sec.append(librosa.get_duration(filename=s))
print(Counter(sizes_sec))

/tmp/ipykernel_173815/2799101307.py:13: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  sizes_sec.append(librosa.get_duration(filename=s))


Counter({240.0: 99, 39.24959375: 1})


In [31]:
240 / 5

48.0

In [37]:
import torchaudio

sizes_sec = []
for s in soundscapes:
    data, sr = torchaudio.load(s)
    sizes_sec.append(data.shape[1] // 32000)
print(Counter(sizes_sec))

Counter({240: 99, 39: 1})
